In [19]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4"
from transformers import AutoModelForSeq2SeqLM
import torch
from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import default_data_collator
from transformers import AutoTokenizer
from tqdm import tqdm
import pandas as pd
import numpy
import random
from peft.utils.save_and_load import set_peft_model_state_dict, get_peft_model_state_dict
from peft import PeftModel, PeftConfig
from functools import partial
from typing import List, Optional, Union
import copy
from transformers import LlamaForCausalLM, LlamaTokenizer
import torch.nn as nn
from peft.tuners.lora.layer import LoraLayer
import types
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from transformers import TrainingArguments
#from utils.prompter import Prompter

In [ ]:
import os
import torch
import json
import numpy as np
from datasets import load_dataset
from transformers import (
    LlamaForCausalLM, 
    LlamaTokenizer,
    TrainingArguments
)
from peft import (
    LoraConfig, 
    get_peft_model, 
    prepare_model_for_kbit_training,
    TaskType
)
from trl import SFTTrainer
#from utils.prompter import Prompter

from transformers import AutoModelForSeq2SeqLM
import torch
from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import default_data_collator
from transformers import AutoTokenizer
from tqdm import tqdm
import pandas as pd
import numpy
import random
from peft.utils.save_and_load import set_peft_model_state_dict, get_peft_model_state_dict
from peft import PeftModel, PeftConfig
from functools import partial
from typing import List, Optional, Union
import copy

import torch
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
import seaborn as sns

tokenizer = None
def load_base_model_and_lora_modules(lora_module_list: List[str], model_name_or_path: Optional[str] = None):
    """load base model and lora modules from huggingface model hub

    Args:
        lora_module_list (List[str]): a list of lora module names available in huggingface model hub
        model_name_or_path (Optional[str]): base model name, default is None
    """
    # use gpu if available
    device = "cuda" if torch.cuda.is_available() else "cpu"
    # load basic model
    default_peft_model_id = lora_module_list[0]
    # find the base model
    if model_name_or_path is None:
        model_name_or_path = PeftConfig.from_pretrained(default_peft_model_id).base_model_name_or_path
    global tokenizer    
    tokenizer = LlamaTokenizer.from_pretrained(model_name_or_path)
    tokenizer.pad_token_id = 0
    tokenizer.padding_side = "left"
    tokenizer.pad_token = tokenizer.bos_token

    base_model = LlamaForCausalLM.from_pretrained(
        model_name_or_path, torch_dtype=torch.float16
    )
    base_model.bfloat16()
    # 0 is the default model
    try:
        peft_model = PeftModel.from_pretrained(base_model, default_peft_model_id)
    except:
        raise Exception(f'{default_peft_model_id} is unable to load into the model {model_name_or_path}')
        
    peft_model = peft_model.to(device)
    peft_model.eval()

    print("> Begin to load lora modules")
    cache = {}

    first_dict = None

    for peft_model_id in tqdm(lora_module_list):
        print("> Loading {} ...".format(peft_model_id))
        cur_peft_model = PeftModel.from_pretrained(base_model, peft_model_id)
        cache[peft_model_id] = copy.deepcopy(get_peft_model_state_dict(cur_peft_model))

        if first_dict is None:
            first_dict = cache[peft_model_id]
        # check whether the LoRA can be merged into one 
        try:
            # detect whether the arch is the same
            for key in first_dict.keys():
                assert first_dict[key].shape == cache[peft_model_id][key].shape
        except:
            raise Exception(f'LoRA Modules {peft_model_id} cannot be merged since it has a different arch (e.g., rank).')
               
    return peft_model, tokenizer, cache


In [ ]:
def initialize_trainer(used_lora_list):
    #lora_list = ['story_cloze', 'hellaswag', 'piqa', 'copa', 'sst2', 'cb', 'multirc', 'wnli', 'squad_v2', 'web_nlg_en', 'definite_pronoun_resolution', 'wmt16_translate_tren', 'wmt16_translate_deen', 'squad_v1', 'dart', 'cosmos_qa', 'mnli_matched', 'anli_r3', 'e2e_nlg', 'anli_r2', 'natural_questions', 'paws_wiki', 'wmt16_translate_ruen', 'glue_qqp', 'story_cloze', 'openbookqa', 'yelp_polarity_reviews', 'arc_easy', 'wmt16_translate_fien', 'anli_r1', 'mnli_mismatched', 'imdb_reviews', 'wmt16_translate_roen', 'common_gen', 'snli', 'sentiment140', 'piqa', 'wmt14_enfr', 'wsc', 'arc_challenge', 'copa', 'qnli', 'glue_mrpc', 'bool_q', 'hellaswag', 'wmt16_translate_csen', 'rte', 'drop', 'record', 'stsb', 'trivia_qa', 'para_crawl_enes', 'quac', 'wiki_lingua_english_en', 'aeslc', 'gigaword', 'trec', 'cnn_dailymail', 'opinion_abstracts_idebate', 'samsum', 'opinion_abstracts_rotten_tomatoes', 'multi_news', 'true_case', 'fix_punct', 'cola', 'word_segment', 'wic', 'ag_news_subset', 'math_dataset', 'coqa']
    model_path = "meta-llama/Llama-2-7b-hf"
    #num_lora_modules = 2
    used_lora_list = used_lora_list
    lora_module_list = [f"Styxxxx/llama2_7b_lora-{name}" for name in used_lora_list]

    model, tokenizer, lora_cache = load_base_model_and_lora_modules(lora_module_list, model_path)
    return model, tokenizer, lora_cache

In [29]:
split_list = [3, 4, 2, 10, 3, 2, 6, 4, 4, 7]

In [ ]:
lora_list = [
    'natural_questions', 'arc_challenge', 'arc_easy', 'story_cloze', 'piqa', 'copa',
    'hellaswag', 'definite_pronoun_resolution', 'wsc', 'cb', 'wnli', 'mnli_matched',
    'anli_r1', 'anli_r2', 'anli_r3', 'mnli_mismatched', 'snli', 'qnli', 'rte',
    'paws_wiki', 'glue_qqp', '', 'cosmos_qa', 'record', 'multirc', 'squad_v1',
    'squad_v2', 'openbookqa', 'bool_q', 'drop', 'sst2', 'yelp_polarity_reviews',
    'imdb_reviews', 'sentiment140', 'web_nlg_en', 'dart', 'e2e_nlg', 'common_gen',
    'wmt16_translate_deen', 'wmt16_translate_fien', 'wmt16_translate_roen',
    'wmt16_translate_ruen', 'wmt16_translate_tren', 'wmt16_translate_csen',
    'wmt14_enfr', 'wmt16_translate_roen'
]

In [ ]:
'''
3, [answering comprehension question from wikipedia/ science multiple choice] [why should these be connected?]
4, [story ending / answering common sense questino / common sense causal reasoning / Intuitive reasoning that is hard for AI]
2, [figure out what the pronoun refers to / similar pronoun problem]
10, [] natural language inference. Trying to figure out the relationships between two sentences, two blocks of texts (till rte)
3, [similar words different meaning/ question paraphrase pair / comprehension mmc???]
2, [large-scale reading comprehension / Reading comprehension on large texts]
6, []Answer question using a large textbook/ corpus, 
4, [] Sentiment analysis 
4, [] generating text with entity relation pairs
7. [] Language translation
'''

In [26]:
!pip install seaborn

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)


In [22]:
!pip install scikit-learn

  Using cached scikit_learn-1.6.1-cp312-cp312-macosx_12_0_arm64.whl.metadata (31 kB)
  Using cached scipy-1.15.2-cp312-cp312-macosx_14_0_arm64.whl.metadata (61 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.6.1-cp312-cp312-macosx_12_0_arm64.whl (11.2 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached scipy-1.15.2-cp312-cp312-macosx_14_0_arm64.whl (22.4 MB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)


In [9]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 5.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 11.1 MB/s eta 0:00:00


In [11]:
!pip install trl

  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 5.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.2/243.2 kB 15.4 MB/s eta 0:00:00
Using cached markdown_it_py-3.0.0-py3-none-any.whl (87 kB)
Using cached mdurl-0.1.2-py3-none-any.whl (10.0 kB)


In [16]:
!pip install utils